# Rule: **prepare_sector_network**

**Outputs**

- results/prenetworks/`elec_s{simpl}_{clusters}_l{ll}_{opts}_{sector_opts}_{horizons}.nc`

In [ ]:
######################################## Parameters

### Run
name = 'co2_05'
prefix = 'add'

### Network
simpl = ''
clusters = 100
ll = 'v1.0'
opts = ''
sector_opts = ''
horizons = '2030'

In [ ]:
##### Import packages
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import yaml
import os 
import sys
import numpy as np


##### Import local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import functions as xp


##### Read params.yaml
with open('../params.yaml', 'r') as configfile:
    params = yaml.safe_load(configfile)


##### Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


##### Region files
file_regions_onshore = f'regions_onshore_elec_s{simpl}_{clusters}.geojson'
file_regions_offshore = f'regions_offshore_elec_s{simpl}_{clusters}.geojson'
path_regions = f'{params['rootpath']}/resources/{prefix}/{name}/'
gdf_regions_onshore = gpd.read_file(path_regions+file_regions_onshore)
gdf_regions_offshore = gpd.read_file(path_regions+file_regions_offshore)


##### NUTS files (must contain at least columns 'NUTS_ID' and 'geometry')
file_NUTS2 = 'NUTS2_ES.geojson'
file_NUTS3 = 'NUTS3_ES.geojson'
path_NUTS = f'{params['rootpath']}/data_ES/nuts/'
gdf_NUTS2 = gpd.read_file(path_NUTS+file_NUTS2)
gdf_NUTS3 = gpd.read_file(path_NUTS+file_NUTS3)


## `elec_s{simpl}_{clusters}_l{ll}_{opts}_{sector_opts}_{horizons}.nc`

Load the network and show its components.

In [ ]:
file = f'elec_s{simpl}_{clusters}_l{ll}_{opts}_{sector_opts}_{horizons}.nc'
path = f'{params['rootpath']}/results/{prefix}/{name}/prenetworks/'

n = pypsa.Network(path+file)

n

Plot the network.

In [ ]:
#################### Parameters
line_widths = 1*n.lines.s_nom / 1e3



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, line_widths=line_widths, bus_sizes=params['bus_sizes'], bus_colors=params['bus_colors'], boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Variable: `n.buses`

Five buses related to carriers **none**, **gas**, **co2**, **co2_stored** and **co2_sequestered** have been included.

Place `n.buses` in a dataFrame and show its content.

In [ ]:
bb = n.buses

bb.head()

### Variable: `n.carriers`

Several carriers may have been added (H2 Store, OCGT, co2, co2_sequestered, co2_stored, electricity, gas, none).

Place `n.carriers` in a dataFrame and show its content.

In [ ]:
cc = n.carriers

cc.head()

### Variable: `n.generators`

The number of generators has been reduced because CCGT and OCGT have been represented with links.

Place `n.generators` in a dataFrame and show its content.

In [ ]:
gg = n.generators

gg.head()

#### Summary

What is the aggregated capacity per carrier? 

How many buses have generators for each carrier? How many of them have zero capacity?

In [ ]:
gg.groupby('carrier').agg(
    Total_capacity=pd.NamedAgg(column='p_nom', aggfunc='sum'),
    Buses=pd.NamedAgg(column='p_nom', aggfunc='size'),
    Buses_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x == 0).sum()),
    Buses_non_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x != 0).sum()),
)

#### Costs

Cost may have changed for renewable technologies.. why? grid connection costs??? (<<<CHECK)

What is the capital cost of generators?

In [ ]:
#################### Parameters

### Select carriers
carrier_list = ['onwind', 'solar', 'offwind-float']



#################### Figure
fig_size = [10,5]
fig, ax = plt.subplots(1,1,figsize=fig_size)

# Give a message with the carriers not considered
carrier_all = gg['carrier'].unique().tolist()
carrier_excluded = [carrier for carrier in carrier_all if carrier not in carrier_list]
print(f'Carriers ommitted: {carrier_excluded}')

# Define bins
minimo = 0 # 0.95 * gg.loc[ gg['carrier'].isin(carrier_list), 'capital_cost'].min()
maximo = 1.05 * gg.loc[ gg['carrier'].isin(carrier_list), 'capital_cost'].max()
ancho = 1000

bins = np.arange(minimo, maximo, 1000)

# Define colors
tech_colors = n.carriers['color']


for carrier in carrier_list:

    ##### Filter the carrier
    df = gg[gg['carrier']==carrier]


    ##### Only single cost for the carrier
    if df['capital_cost'].round(2).nunique()==1:

        valor = df['capital_cost'].unique()[0]

        ax.axvline(x=valor, label=carrier, color = tech_colors[carrier])

        print(f'Capital cost for {carrier} is: {valor:.2f} EUR/MW·year')

    ##### Different costs for the carrier
    else:
        plt.hist(df['capital_cost'], bins=bins, 
                 edgecolor='none', color = tech_colors[carrier],
                 label=carrier, alpha=1)
        
        valor = df['capital_cost'].mean()
        print(f'Average capital cost for {carrier} is: {valor:.2f} EUR/MW·year')


ax.set_title('capital cost')
ax.set_xlabel('EUR/(MW·year)')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.5)

### Variable: `n.global_constraints`

The constraint related to co2 emissions has been added.

Place `n.global_constraints` in a dataFrame and show its content.

In [ ]:
gc = n.global_constraints

gc

### Variable: `n.links`

Changes:

- Links related to CCGT and OCGT have been included.

- Interconnections have been reversed.

- Values in columns 'p_nom_max' have been updated.

- 4 more columns have been added (<<< DESCRIBE).


Place `n.links` in a dataFrame and show its content.

In [ ]:
lk = n.links

lk.head()

### Variable: `n.stores`

Stores related to **co2_atmosphere**, **co2_sequestered**, **co2_stored** and **gas_stored** have been added.

Place `n.stores` in a dataFrame and show its content.

In [ ]:
st = n.stores

st.head()

### Analysis: Interconnections (`PyPSA-Spain` only)

In `PyPSA-Spain`, interconnections with FR and PT are included at this stage.

Each interconnection is represented through a new link with a new bus. The bus is located in the border, and includes a generator and a load. The marginal cost of the generator represents the market price in FR or PT.

What are the main characteristics of DC links (interconnections and the existing ones)?

In [ ]:
lk_DC = lk.loc[lk['carrier']=='DC']

print(lk_DC[['p_nom', 'length', 'capital_cost', 'p_nom_extendable']])